In [ ]:
!pip -q install transformers
!pip -q install datasets
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
import pandas as pd
from torch import cuda
from tabulate import tabulate
from torch.utils.data import Dataset
from transformers import BertTokenizerFast
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from torch import Tensor
import copy
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.8 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
from transformers import BertForSequenceClassification
model0 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vo

In [ ]:
#get cluster index and eda, back translation data, need to run eda.ipynb first
cluster_index = [0 for i in range(6)]
cluster_index[0] = pd.read_pickle('/content/drive/MyDrive/cluster_0.pkl').index.tolist()
cluster_index[1] = pd.read_pickle('/content/drive/MyDrive/cluster_1.pkl').index.tolist()
cluster_index[2] = pd.read_pickle('/content/drive/MyDrive/cluster_2.pkl').index.tolist()
cluster_index[3] = pd.read_pickle('/content/drive/MyDrive/cluster_3.pkl').index.tolist()
cluster_index[4] = pd.read_pickle('/content/drive/MyDrive/cluster_4.pkl').index.tolist()
cluster_index[5] = pd.read_pickle('/content/drive/MyDrive/cluster_4.pkl').index.tolist()
rd_text = pd.read_pickle('/content/drive/MyDrive/rd_all.pkl')['text'].tolist()
rd_label = pd.read_pickle('/content/drive/MyDrive/rd_all.pkl')['coarse_label'].tolist()
ri_text = pd.read_pickle('/content/drive/MyDrive/ri_all.pkl')['text'].tolist()
ri_label = pd.read_pickle('/content/drive/MyDrive/ri_all.pkl')['coarse_label'].tolist()
sr_text = pd.read_pickle('/content/drive/MyDrive/sr_all.pkl')['text'].tolist()
sr_label = pd.read_pickle('/content/drive/MyDrive/sr_all.pkl')['coarse_label'].tolist()
rs_text = pd.read_pickle('/content/drive/MyDrive/rs_all.pkl')['text'].tolist()
rs_label = pd.read_pickle('/content/drive/MyDrive/rs_all.pkl')['coarse_label'].tolist()
de_text = pd.read_pickle('/content/drive/MyDrive/de_aug_all.pkl')['text'].tolist()
de_label = pd.read_pickle('/content/drive/MyDrive/de_aug_all.pkl')['coarse_label'].tolist()
ru_text = pd.read_pickle('/content/drive/MyDrive/ru_aug_all.pkl')['text'].tolist()
ru_label = pd.read_pickle('/content/drive/MyDrive/ru_aug_all.pkl')['coarse_label'].tolist()

## Dataset and Dataloader

In [ ]:
# preparing dataset
class TrecDataset(Dataset):
    def __init__(self, texts, label, tokenizer):
        self.texts = texts
        self.labels = label
        self.tokenizer = tokenizer
        self.encodings = self.tokenizer(self.texts, truncation=True, padding=True)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
dataset = load_dataset('trec')

train_texts = dataset['train']['text']
train_labels = list(dataset['train']['coarse_label'])

test_texts = dataset['test']['text']
test_labels = list(dataset['test']['coarse_label'])

print('len(train):', len(train_texts))
print('len(test):', len(test_texts))

Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

len(train): 5452
len(test): 500


In [ ]:
# create dataset with BERT being the tokenizer and baseline model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)

train_dataset1 = TrecDataset(train_texts, train_labels, tokenizer)
test_dataset1 = TrecDataset(test_texts, test_labels, tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#Base line

In [ ]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='/content/drive/MyDrive/model1',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=1,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    # TensorBoard log directory
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="epoch",
    # fp16=False if device == "cpu" else True,
    fp16=False,
    load_best_model_at_end=False
)
print("training arguments updated...")
pred_test = 0
def compute_metrics(pred):
    label = pred.label_ids
    pred_test = pred
    preds = pred.predictions.argmax(-1)#np.argmax(pred, axis=-1)#如果是bert训练这里需要加个[0]
    precision, recall, f1, _ = precision_recall_fscore_support(label, preds, average='macro', warn_for=tuple(), zero_division=1)
    acc = accuracy_score(label, preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

# create trainer object
trainer = Trainer(
    model=model0,
    # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataset1,#_aug,#train_dataset_em_augtrain_dataset_aug,#train_em_set,#train_dataset,
    eval_dataset=test_dataset1,#test_dataset,
    compute_metrics=compute_metrics
)
#model2.to('cuda')
# start model training
trainer.train()
test_results = trainer.evaluate(eval_dataset=test_dataset1)
print('Test Accuracy:', round(test_results['eval_Accuracy'], 4))
print('Test F1:', round(test_results['eval_F1'], 4))
print('Test Recall:', round(test_results['eval_Recall'], 4))

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5452
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1023
  Number of trainable parameters = 109486854


training arguments updated...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.757200,1.558977,0.384000,0.175928,0.624723,0.305128
100,1.157400,0.504470,0.860000,0.716602,0.916443,0.721322
150,0.529300,0.264560,0.928000,0.780169,0.946546,0.783035
200,0.290600,0.216721,0.940000,0.822240,0.954457,0.812722
250,0.259800,0.184899,0.958000,0.930507,0.961925,0.911585
300,0.269100,0.387447,0.908000,0.893687,0.946064,0.867556
350,0.208600,0.147621,0.964000,0.948688,0.970136,0.932515
400,0.136600,0.202949,0.952000,0.940134,0.965294,0.922116
450,0.140000,0.241029,0.950000,0.927490,0.964860,0.903598
500,0.188300,0.146252,0.962000,0.928628,0.924268,0.934288


***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
Saving model checkpoint to /content/drive/MyDrive/model1/checkpoint-341
Configuration saved in /content/drive/MyDrive/model1/checkpoint-341/config.json
Model weights saved in /content/drive/MyDrive/model1/checkpoint-341/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***

NameError: ignored

In [ ]:
predictions, labels, metrics = trainer.predict(test_dataset=test_dataset1)
pred_labels = []
for i in predictions:
  label = np.argmax(i)
  pred_labels.append(label)
misclass_index = np.where(np.array(pred_labels) != test_labels)[0].tolist()
print(misclass_index)

***** Running Prediction *****
  Num examples = 500
  Batch size = 1


[99, 133, 135, 190, 223, 234, 241, 249, 251, 259, 291, 325, 361, 425, 460]


#Train model with sentence level augmentation

In [ ]:
#model bertsequenceclassification
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import BertConfig
from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertLayer, BertEmbeddings, BertPooler
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.init_weights()
        self.bert = model0.bert
        self.dropout = model0.dropout#nn.Dropout(config.hidden_dropout_prob)
        self.classifier = model0.classifier#nn.Linear(config.hidden_size, config.num_labels)
         # Init classifier weights according to initialization rules of model
        #model2._init_weights()

        #self.init_weights() = 
        #model2.init_weights()

    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = torch.nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = torch.nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        #if not return_dict:
        #    output = (logits,) + outputs[2:]
        #    return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            #hidden_states=outputs.hidden_states,
            #attentions=outputs.attentions,
        )
bert_config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=False,num_labels=6)

model = BertForSequenceClassification(bert_config)

In [ ]:
def mix_aug(rs,sr,rd,ri,bt_de,bt_ru,index):
  dataset = load_dataset('trec')

  train_texts = dataset['train']['text']
  train_labels = list(dataset['train']['coarse_label'])
  #train_embedding = pd.read_pickle("/content/drive/MyDrive/train.pkl")['texts'].tolist()
  aug_text = copy.deepcopy(train_texts)
  aug_label = copy.deepcopy(train_labels)
  #aug_embedding = []
  #train_embedding_aug_label = copy.deepcopy(train_labels)
  #train_embedding_aug = copy.deepcopy(train_embedding)
  #aug_embedding_label = []
  #sample_embedding = [0 for i in range(len(index))]
  
  all_cluster = True
  if index !=-1:
    all_cluster = False
  if rs ==1 :
    if all_cluster:
      for i in range(len(rs_label)):
        aug_text.append(rs_text[i])
        aug_label.append(rs_label[i])
    else:
      for i in range(len(index)):
        aug_text.append(rs_text[index[i]])
        aug_label.append(rs_label[index[i]])
  if rd ==1 :
    if all_cluster:
      for i in range(len(rd_label)):
        aug_text.append(rd_text[i])
        aug_label.append(rd_label[i])
    else:
      for i in range(len(index)):
        aug_text.append(rd_text[index[i]])
        aug_label.append(rd_label[index[i]])
  if ri ==1 :
    if all_cluster:
      for i in range(len(ri_label)):
        aug_text.append(ri_text[i])
        aug_label.append(ri_label[i])
    else:
      for i in range(len(index)):
        aug_text.append(ri_text[index[i]])
        aug_label.append(ri_label[index[i]])
  if sr ==1 :
    if all_cluster:
      for i in range(len(sr_label)):
        aug_text.append(sr_text[i])
        aug_label.append(sr_label[i])
    else:
      for i in range(len(index)):
        aug_text.append(sr_text[index[i]])
        aug_label.append(sr_label[index[i]])
  if bt_de ==1 :
    if all_cluster:
      for i in range(len(de_label)):
        aug_text.append(de_text[i])
        aug_label.append(de_label[i])
    else:
      for i in range(len(index)):
        aug_text.append(de_text[index[i]])
        aug_label.append(de_label[index[i]])
  if bt_ru ==1 :
    if all_cluster:
      for i in range(len(ru_label)):
        aug_text.append(ru_text[i])
        aug_label.append(ru_label[i])
    else:
      for i in range(len(index)):
        aug_text.append(ru_text[index[i]])
        aug_label.append(ru_label[index[i]])
  #print(len(aug_embedding))
  #print(len(aug_embedding_label)) 
  #print(len(aug_text))
  #print(len(aug_label))
  text_aug_df = pd.DataFrame(columns=['texts', 'labels'])
  text_aug_df['texts'] = aug_text
  text_aug_df['labels'] = aug_label
  #TrecDataset(train_texts, train_labels, tokenizer)
  print("the augmented text trainset's length is:",len(text_aug_df))
  train_dataset_aug =  TrecDataset(aug_text, aug_label, tokenizer)#Dataset.from_pandas(text_aug_df)

  return train_dataset_aug

In [ ]:
train_dataset_aug = mix_aug(1,1,1,1,1,1,misclass_index)

  0%|          | 0/2 [00:00<?, ?it/s]

the augmented text trainset's length is: 5542


In [ ]:
from transformers import BertConfig
#initialize the model with bert_uncased config
#encoder = model1.bert
# updating training arguments
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='/content/drive/MyDrive/model_sentence_aug',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=1,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    # TensorBoard log directory
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="epoch",
    # fp16=False if device == "cpu" else True,
    fp16=False,
    load_best_model_at_end=False
)
print("training arguments updated...")
pred_test = 0
def compute_metrics(pred):
    label = pred.label_ids
    pred_test = pred
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(label, preds, average='macro', warn_for=tuple(), zero_division=1)
    acc = accuracy_score(label, preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

# create trainer object
trainer = Trainer(
    model=model,
    # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataset_aug,#_aug,#train_dataset_em_augtrain_dataset_aug,#train_em_set,#train_dataset,
    eval_dataset=test_dataset1,#test_dataset,
    compute_metrics=compute_metrics
)
#model2.to('cuda')
model.to('cuda')
# start model training
trainer.train()
test_results = trainer.evaluate(eval_dataset=test_dataset1)
print("Result after sentence-level augmentation:")
print('Test Accuracy:', round(test_results['eval_Accuracy'], 4))
print('Test F1:', round(test_results['eval_F1'], 4))
print('Test Recall:', round(test_results['eval_Recall'], 4))

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12082
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2268
  Number of trainable parameters = 109486854


training arguments updated...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.236200,0.114603,0.970000,0.965013,0.976217,0.955638
100,0.181900,0.135320,0.970000,0.943507,0.976581,0.922106
150,0.224800,0.133721,0.966000,0.959642,0.969310,0.952273
200,0.211800,0.149393,0.960000,0.944970,0.964263,0.931117
250,0.141300,0.143098,0.960000,0.925497,0.903800,0.968836
300,0.217400,0.133103,0.964000,0.961033,0.958088,0.965437
350,0.196000,0.109993,0.970000,0.965424,0.977311,0.955597
400,0.149800,0.094524,0.978000,0.979888,0.981546,0.978567
450,0.146400,0.138725,0.972000,0.957295,0.978163,0.942026
500,0.189800,0.135747,0.968000,0.964473,0.958892,0.972536


***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  N

Result after sentence-level augmentation:
Test Accuracy: 0.98
Test F1: 0.9751
Test Recall: 0.9818


#Train model with embedding level augmentation

Build model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self, config):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(768, 6)
        self.dropout = model0.dropout#
        self.classifier = model0.classifier

    def forward(self,labels=None,texts=None):
        #embeddings = torch.from_numpy(texts)
        #print(texts)
        pooled_output = self.dropout(texts)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if 6 == 1:
                #  We are doing regression
                loss_fct = torch.nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = torch.nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, 6), labels.view(-1))

    

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
        )

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import itertools
encoder = model.bert
def get_neighbour(embedding_list,train_embedding,index,k):
  sim_index = [0 for i in range(len(embedding_list))]
  train_sim = [[] for i in range(len(embedding_list))]
  if index == -1:
    train_embedding_copy = train_embedding
  train_embedding_copy = np.array(train_embedding)[index]
  cos_sim = cosine_similarity(embedding_list, train_embedding_copy)#shape is len(embedding_list),len(train_embedding)
  for i in range(len(sim_index)):
    sim_index[i] = np.argsort(np.array(cos_sim[i]))[-k:]
  for i in range(len(sim_index)):
    for j in range(len(sim_index[i])):
      train_sim[i].append(train_embedding[sim_index[i][j]])

  return train_sim#len of misclass, each element is a list

def get_aug(embedding_list):
  cc =[0 for i in range(len(embedding_list))]
  for i in range(len(embedding_list)):
    cc[i] = list(itertools.combinations(embedding_list[i], 2))
  embedding_list = [[] for i in range(len(embedding_list))]
  for i in range(len(cc)):
    for j in range(len(cc[i])):
      embedding_list[i].append(np.mean(np.array([cc[i][j][0], cc[i][j][1]]), axis=0))
  
  return embedding_list

def em_aug(index,k,emd=1):
  dataset = load_dataset('trec')

  train_texts = dataset['train']['text']
  train_labels = list(dataset['train']['coarse_label'])
  train_embedding = [0 for i in range(len(train_texts))]#pd.read_pickle("/content/drive/MyDrive/train.pkl")['texts'].tolist()
  for i in range(len(train_texts)):
    train_embedding[i] = encoder(**tokenizer(train_texts[i],return_tensors="pt").to('cuda'))[1]
    train_embedding[i] = Tensor.cpu(train_embedding[i]).detach().numpy().flatten()
  #aug_text = copy.deepcopy(train_texts)
  aug_label = copy.deepcopy(train_labels)
  aug_embedding = []
  train_embedding_aug_label = copy.deepcopy(train_labels)
  train_embedding_aug = copy.deepcopy(train_embedding)
  aug_embedding_label = []
  sample_embedding = [0 for i in range(len(index))]
  if emd ==1:
    for i in range(len(index)):
      sample_embedding[i] = encoder(**tokenizer(train_texts[index[i]],return_tensors="pt").to('cuda'))[1]#.detach().numpy().flatten()#pt后面括号有tocuda
      sample_embedding[i] = Tensor.cpu(sample_embedding[i]).detach().numpy().flatten()#sample_embedding[i].detach().numpy().flatten()#
    nearest_neighbour = get_neighbour(sample_embedding,train_embedding,index,k)
    aug_embedding = get_aug(nearest_neighbour)
    #j=0
    for i in range(len(aug_embedding)):
      for j in range(len(aug_embedding[i])):
        aug_embedding_label.append(train_labels[index[i]])      
    for i in range(len(aug_embedding)):
      for j in range(len(aug_embedding[i])):
        train_embedding_aug.append(aug_embedding[i][j])
        train_embedding_aug_label.append(aug_embedding_label[i])
  embedding_aug_df = pd.DataFrame(columns=['texts', 'labels'])
  embedding_aug_df['texts'] = train_embedding_aug
  embedding_aug_df['labels'] = train_embedding_aug_label
  print("the augmented text trainset's length is:",len(embedding_aug_df))
  train_dataset_em_aug = Dataset.from_pandas(embedding_aug_df)

  return train_dataset_em_aug

In [ ]:
train_dataset_em_aug = em_aug([2,3,4],5,emd=1)

  0%|          | 0/2 [00:00<?, ?it/s]

the augmented text trainset's length is: 5461


Build model

In [ ]:
from transformers import BertConfig
#initialize the model with bert_uncased config
bert_config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=False,num_labels=6)
classifier = Classifier(bert_config)
# updating training arguments
test_texts = dataset['test']['text']
test_labels = list(dataset['test']['coarse_label'])
test_embedding = [0 for i in range(len(test_texts))]
for i in range(len(test_texts)):
  test_embedding[i] = encoder(**tokenizer(test_texts[i],return_tensors="pt").to('cuda'))[1]
  test_embedding[i] = Tensor.cpu(test_embedding[i]).detach().numpy().flatten()
test_em_df = pd.DataFrame(columns=['texts', 'labels'])
test_em_df['texts'] = test_embedding
test_em_df['labels'] = test_labels
test_dataset_em = Dataset.from_pandas(test_em_df)

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='/content/drive/MyDrive/model1',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=1,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    # TensorBoard log directory
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="epoch",
    # fp16=False if device == "cpu" else True,
    fp16=False,
    load_best_model_at_end=False
)
print("training arguments updated...")
pred_test = 0
def compute_metrics(pred):
    label = pred.label_ids
    pred_test = pred
    preds = pred.predictions.argmax(-1)#np.argmax(pred, axis=-1)#如果是bert训练这里需要加个[0]
    precision, recall, f1, _ = precision_recall_fscore_support(label, preds, average='macro', warn_for=tuple(), zero_division=1)
    acc = accuracy_score(label, preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

# create trainer object
trainer = Trainer(
    model=classifier,
    # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataset_em_aug,#_aug,#train_dataset_em_augtrain_dataset_aug,#train_em_set,#train_dataset,
    eval_dataset=test_dataset_em,#test_dataset,
    compute_metrics=compute_metrics
)
#model2.to('cuda')
# start model training
trainer.train()
test_results = trainer.evaluate(eval_dataset=test_dataset_em)
print('Test Accuracy:', round(test_results['eval_Accuracy'], 4))
print('Test F1:', round(test_results['eval_F1'], 4))
print('Test Recall:', round(test_results['eval_Recall'], 4))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vo

training arguments updated...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.041400,0.154891,0.972000,0.966601,0.978173,0.957356
100,0.008500,0.156394,0.972000,0.966601,0.978173,0.957356
150,0.008900,0.158353,0.972000,0.966601,0.978173,0.957356
200,0.009200,0.158871,0.972000,0.966601,0.978173,0.957356
250,0.019000,0.159523,0.972000,0.966601,0.978173,0.957356
300,0.022600,0.158310,0.972000,0.966601,0.978173,0.957356
350,0.026300,0.157893,0.972000,0.966601,0.978173,0.957356
400,0.013000,0.158830,0.972000,0.966601,0.978173,0.957356
450,0.028400,0.157890,0.972000,0.966601,0.978173,0.957356
500,0.033600,0.155961,0.972000,0.966601,0.978173,0.957356


***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
Saving model checkpoint to /content/drive/MyDrive/model1/checkpoint-342
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
***** Running Evaluation *****

Test Accuracy: 0.202
Test F1: 0.1499
Test Recall: 0.2674


In [ ]:
test_results = trainer.evaluate(eval_dataset=test_dataset_em)
print('Test Accuracy:', round(test_results['eval_Accuracy'], 4))
print('Test F1:', round(test_results['eval_F1'], 4))
print('Test Recall:', round(test_results['eval_Recall'], 4))

***** Running Evaluation *****
  Num examples = 500
  Batch size = 1


Test Accuracy: 0.972
Test F1: 0.9666
Test Recall: 0.9574
